In [1]:
!pip install -U spacy
!python -m spacy download en_core_web_sm
!pip install -U gensim==3.8.3
!pip install transformers==4.10.2
!pip install sentencepiece==0.1.96


     |████████████████████████████████| 6.0 MB 4.2 MB/s 
     |████████████████████████████████| 628 kB 43.7 MB/s 
     |████████████████████████████████| 451 kB 50.3 MB/s 
     |████████████████████████████████| 10.1 MB 39.4 MB/s 
     |████████████████████████████████| 181 kB 49.5 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 13.9 M

In [12]:
import spacy 
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

In [11]:
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords

In [14]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
tokenizer = AutoTokenizer.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")

In [15]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)

device  cuda


In [16]:
stopwords = list(STOP_WORDS)
print(stopwords)

['also', 'used', 'often', 'give', 'other', 'becoming', 'first', 'per', 'became', 'why', 'before', 'hence', '‘ve', 'others', '’m', 'no', 'what', 'thru', 'unless', 'does', 'three', 'their', 'than', 'within', 'already', 'neither', 'after', 'everywhere', 'twenty', 'now', 'sixty', 'wherein', 'whatever', "'m", '’re', 'noone', 'herein', 'her', 'many', 'say', 'is', 'thereby', 'herself', 'top', 'everyone', 'afterwards', 'ten', 'too', 'name', '‘re', 'ca', 'somewhere', 'himself', 'anyone', 'move', 'quite', 'that', 'go', 'rather', 'whoever', 'all', 'beyond', "'re", 'we', 'behind', 'besides', 'indeed', 'toward', 'your', 'another', 'amount', 'least', 'become', 'very', 'thereupon', 'above', 'due', 'own', 'less', 'here', 'whereupon', 'fifteen', 'nobody', 'someone', 'while', 'at', 'could', 'since', 'towards', 'never', 'somehow', 'might', 'ours', "'ve", 'elsewhere', 'although', 'therein', 'him', 'full', 'in', 'us', 'therefore', 'until', 'anywhere', 'few', 'more', 'eleven', 'last', 'show', 'not', 'throug

In [17]:
nlp = spacy.load('en_core_web_sm')

In [18]:
text = input()

[ ] print(summarize(text, ratio=1)) Elon Musk is working to revolutionize transportation both on Earth, through electric car maker Tesla -- and in space, via rocket producer SpaceX. He owns 23% of Tesla but has pledged more than half his stake as collateral for loans; Forbes has discounted his stake to take the loans into account. The Tesla CEO stepped down as chairman in 2018, after making alleged "false statements" about a plan to take Tesla private, triggering an SEC probe. SpaceX, Musk's rocket company, is valued at $74 billion after its latest funding round in February 2021. He grew up in South Africa, then immigrated to Canada at age 17. He landed in the U.S. as a transfer student to the University of Pennsylvania.


In [19]:
print(summarize(text, ratio=1))

[ ] print(summarize(text, ratio=1)) Elon Musk is working to revolutionize transportation both on Earth, through electric car maker Tesla -- and in space, via rocket producer SpaceX.
He owns 23% of Tesla but has pledged more than half his stake as collateral for loans; Forbes has discounted his stake to take the loans into account.
The Tesla CEO stepped down as chairman in 2018, after making alleged "false statements" about a plan to take Tesla private, triggering an SEC probe.
SpaceX, Musk's rocket company, is valued at $74 billion after its latest funding round in February 2021.
He grew up in South Africa, then immigrated to Canada at age 17.
He landed in the U.S. as a transfer student to the University of Pennsylvania.


In [20]:
topics = keywords(text, words=10)

In [21]:
print(topics)

tesla
spacex
funding
alleged
false
car
rocket
private
ceo


In [22]:
doc = nlp(text)

In [ ]:
token = [token.text for token in doc]
print(token)

In [24]:
punctuation = punctuation + '\n'

In [25]:
word_frequency = {}
for word in doc:
    if word.text.lower() not in stopwords:
        if word.text.lower() not in punctuation:
            if word.text not in word_frequency.keys():
                word_frequency[word.text] = 1
            else:
                word_frequency[word.text] += 1

In [26]:
print(word_frequency)

{'print(summarize(text': 1, 'ratio=1': 1, 'Elon': 1, 'Musk': 2, 'working': 1, 'revolutionize': 1, 'transportation': 1, 'Earth': 1, 'electric': 1, 'car': 1, 'maker': 1, 'Tesla': 4, '--': 1, 'space': 1, 'rocket': 2, 'producer': 1, 'SpaceX.': 1, 'owns': 1, '23': 1, 'pledged': 1, 'half': 1, 'stake': 2, 'collateral': 1, 'loans': 2, 'Forbes': 1, 'discounted': 1, 'account': 1, 'CEO': 1, 'stepped': 1, 'chairman': 1, '2018': 1, 'making': 1, 'alleged': 1, 'false': 1, 'statements': 1, 'plan': 1, 'private': 1, 'triggering': 1, 'SEC': 1, 'probe': 1, 'SpaceX': 1, 'company': 1, 'valued': 1, '74': 1, 'billion': 1, 'latest': 1, 'funding': 1, 'round': 1, 'February': 1, '2021': 1, 'grew': 1, 'South': 1, 'Africa': 1, 'immigrated': 1, 'Canada': 1, 'age': 1, '17': 1, 'landed': 1, 'U.S.': 1, 'transfer': 1, 'student': 1, 'University': 1, 'Pennsylvania': 1}


In [27]:
max_frequency = max(word_frequency.values())

In [28]:
max_frequency

4

In [29]:
for word in word_frequency.keys():
    word_frequency[word] = word_frequency[word]/max_frequency

In [30]:
print(word_frequency)

{'print(summarize(text': 0.25, 'ratio=1': 0.25, 'Elon': 0.25, 'Musk': 0.5, 'working': 0.25, 'revolutionize': 0.25, 'transportation': 0.25, 'Earth': 0.25, 'electric': 0.25, 'car': 0.25, 'maker': 0.25, 'Tesla': 1.0, '--': 0.25, 'space': 0.25, 'rocket': 0.5, 'producer': 0.25, 'SpaceX.': 0.25, 'owns': 0.25, '23': 0.25, 'pledged': 0.25, 'half': 0.25, 'stake': 0.5, 'collateral': 0.25, 'loans': 0.5, 'Forbes': 0.25, 'discounted': 0.25, 'account': 0.25, 'CEO': 0.25, 'stepped': 0.25, 'chairman': 0.25, '2018': 0.25, 'making': 0.25, 'alleged': 0.25, 'false': 0.25, 'statements': 0.25, 'plan': 0.25, 'private': 0.25, 'triggering': 0.25, 'SEC': 0.25, 'probe': 0.25, 'SpaceX': 0.25, 'company': 0.25, 'valued': 0.25, '74': 0.25, 'billion': 0.25, 'latest': 0.25, 'funding': 0.25, 'round': 0.25, 'February': 0.25, '2021': 0.25, 'grew': 0.25, 'South': 0.25, 'Africa': 0.25, 'immigrated': 0.25, 'Canada': 0.25, 'age': 0.25, '17': 0.25, 'landed': 0.25, 'U.S.': 0.25, 'transfer': 0.25, 'student': 0.25, 'University':

In [31]:
for key,value in word_frequency.items():
    if key in topics:
        value += 1 
        word_frequency.update({key:value}) 
print(word_frequency.items())

dict_items([('print(summarize(text', 0.25), ('ratio=1', 0.25), ('Elon', 0.25), ('Musk', 0.5), ('working', 0.25), ('revolutionize', 0.25), ('transportation', 0.25), ('Earth', 0.25), ('electric', 0.25), ('car', 1.25), ('maker', 0.25), ('Tesla', 1.0), ('--', 0.25), ('space', 1.25), ('rocket', 1.5), ('producer', 0.25), ('SpaceX.', 0.25), ('owns', 0.25), ('23', 0.25), ('pledged', 0.25), ('half', 0.25), ('stake', 0.5), ('collateral', 0.25), ('loans', 0.5), ('Forbes', 0.25), ('discounted', 0.25), ('account', 0.25), ('CEO', 0.25), ('stepped', 0.25), ('chairman', 0.25), ('2018', 0.25), ('making', 0.25), ('alleged', 1.25), ('false', 1.25), ('statements', 0.25), ('plan', 0.25), ('private', 1.25), ('triggering', 0.25), ('SEC', 0.25), ('probe', 0.25), ('SpaceX', 0.25), ('company', 0.25), ('valued', 0.25), ('74', 0.25), ('billion', 0.25), ('latest', 0.25), ('funding', 1.25), ('round', 0.25), ('February', 0.25), ('2021', 0.25), ('grew', 0.25), ('South', 0.25), ('Africa', 0.25), ('immigrated', 0.25), 

In [32]:
sentence_tokens = [sent for sent  in doc.sents]
print(sentence_tokens)

[[ ] print(summarize(text, ratio=1)), Elon Musk is working to revolutionize transportation both on Earth, through electric car maker Tesla -- and in space, via rocket producer SpaceX. He owns 23% of Tesla but has pledged more than half his stake as collateral for loans; Forbes has discounted his stake to take the loans into account., The Tesla CEO stepped down as chairman in 2018, after making alleged "false statements" about a plan to take Tesla private, triggering an SEC probe., SpaceX, Musk's rocket company, is valued at $74 billion after its latest funding round in February 2021., He grew up in South Africa, then immigrated to Canada at age 17., He landed in the U.S. as a transfer student to the University of Pennsylvania.]


In [33]:
sentence_scores = {}
for sent in sentence_tokens:
    for word in sent:
        if word.text.lower() in word_frequency.keys():
            if sent not in sentence_scores.keys():
                sentence_scores[sent] = word_frequency[word.text.lower()] 
            else:
                sentence_scores[sent] += word_frequency[word.text.lower()]

In [34]:
sentence_scores

{[ ] print(summarize(text, ratio=1)): 0.5,
 Elon Musk is working to revolutionize transportation both on Earth, through electric car maker Tesla -- and in space, via rocket producer SpaceX. He owns 23% of Tesla but has pledged more than half his stake as collateral for loans; Forbes has discounted his stake to take the loans into account.: 9.5,
 The Tesla CEO stepped down as chairman in 2018, after making alleged "false statements" about a plan to take Tesla private, triggering an SEC probe.: 5.75,
 SpaceX, Musk's rocket company, is valued at $74 billion after its latest funding round in February 2021.: 4.5,
 He grew up in South Africa, then immigrated to Canada at age 17.: 1.0,
 He landed in the U.S. as a transfer student to the University of Pennsylvania.: 0.75}

In [35]:
from heapq import nlargest

In [36]:
select_length = int(len(sentence_scores)*0.5)
select_length

3

In [37]:
summary = nlargest(select_length,sentence_scores,key = sentence_scores.get )

In [38]:
summary

[Elon Musk is working to revolutionize transportation both on Earth, through electric car maker Tesla -- and in space, via rocket producer SpaceX. He owns 23% of Tesla but has pledged more than half his stake as collateral for loans; Forbes has discounted his stake to take the loans into account.,
 The Tesla CEO stepped down as chairman in 2018, after making alleged "false statements" about a plan to take Tesla private, triggering an SEC probe.,
 SpaceX, Musk's rocket company, is valued at $74 billion after its latest funding round in February 2021.]

In [39]:
final_summary = [word.text for word in summary ]

In [40]:
summary = ''.join(final_summary)

In [41]:
print(summary)

Elon Musk is working to revolutionize transportation both on Earth, through electric car maker Tesla -- and in space, via rocket producer SpaceX. He owns 23% of Tesla but has pledged more than half his stake as collateral for loans; Forbes has discounted his stake to take the loans into account.The Tesla CEO stepped down as chairman in 2018, after making alleged "false statements" about a plan to take Tesla private, triggering an SEC probe.SpaceX, Musk's rocket company, is valued at $74 billion after its latest funding round in February 2021.


In [42]:

context = summary
text = "paraphrase: "+context + " </s>"
encoding = tokenizer.encode_plus(text,max_length =128, padding=True, return_tensors="pt")
input_ids,attention_mask  = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
model.eval()
diverse_beam_outputs = model.generate(
    input_ids=input_ids,attention_mask=attention_mask,
    max_length=128,
    early_stopping=True,
    num_beams=5,
    num_beam_groups = 5,
    num_return_sequences=5,
    diversity_penalty = 0.70
)
for beam_output in diverse_beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
result = sent[19:]


/usr/local/lib/python3.7/dist-packages/transformers/generation_beam_search.py:197: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect.`max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`,or `group_beam_search(...)`.
  "Passing `max_length` to BeamSearchScorer is deprecated and has no effect."
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2451: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2474: UserWarning: __floordiv__ is deprecated, and its behavior will chang

In [43]:
print(result)

Elon Musk's rocket company is working to revolutionize transportation on Earth, electric car maker Tesla, and space by rocket manufacturer SpaceX; Forbes has discounted his interest in loans to factor in the loans; Musk's rocket company SpaceX has dropped its valuation as chairman in 2018...


In [47]:
!pip install gtts

In [45]:
text = result

In [48]:
from gtts import gTTS


In [49]:
from IPython.display import Audio
tts = gTTS(result)
tts.save('1.wav')
sound = '1.wav'
Audio(sound, autoplay = True)